In [1]:
import tensorflow as tf
import numpy as np
import os
from imutils import paths
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import ResNet50

In [2]:
def preprocess(image):
    img = cv2.cvtColor(image,cv2.COLOR_BGR2BGRA)
    gaussian_3 = cv2.GaussianBlur(img, (0, 0), 2.0)
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5,0,image)
    th, im_th = cv2.thershold(unsharp_image, 0.45 ,0.75 , cv2.THRESH_BINARY)
    return im_th


In [2]:
imagePaths = list(paths.list_images('D://AI ML DL//mask detection//Dataset mask detection//train'))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)
    


In [3]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

In [4]:
baseModel = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)


#(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.10, stratify=labels, random_state=42)
	

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,zoom_range=0.2,
                                   horizontal_flip=True,fill_mode='nearest')

In [7]:
train = train_datagen.flow_from_directory('D:/AI ML DL/mask detection/Dataset mask detection/train/train',target_size=(224,224),batch_size=32, class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation = validation_datagen.flow_from_directory('D:/AI ML DL/mask detection/Dataset mask detection/test/test',target_size=(224,224),batch_size=32, class_mode='binary')

Found 1309 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [8]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network


[INFO] compiling model...


In [ ]:
print("[INFO] training head...")
H = model.fit(
	train_datagen.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

In [9]:
filename="mask_recog.hdf5"
model.save(filename,overwrite=True)